In [8]:
import numpy as np
import pandas as pd

In [9]:
table=pd.read_csv('names.csv')
names=dict(np.array(table))

In [237]:
algebra_operators={-1:'+',-2:'-',-3:'*',-4:'/'}
binar_operators={-6:'='}
unar_operators={-5:'-',-7:'print'}
operands={1:[1,'numb',1],2:[2,'numb',2],3:[3,'numb',3],4:[4.5,'numb',4.5],
          5:['some_string','char','some_string'],6:['a','var',8],7:['b','var',np.NAN]}

In [238]:
stack=[6,2,2,6,-5,-1,-3,-6,-100]
#stack=[6,5,-6,-100]

In [239]:
class parser(object):
    def __init__(self,stack):
        self.stack=stack
        self.count_stack=[]
        self.check_points={}
        self.end_symbols=[-100]
    def show(self):
        print(self.user_variables)
    def count_reversed_form(self,start_point=0):
        stack=self.stack
        i=0
        while stack[i] not in self.end_symbols:
            print(self.count_stack)
            if stack[i] in operands.keys():
                self.count_stack.append([stack[i],'link'])                        
            else:
                if stack[i] in algebra_operators:
                    operation=algebra_operators[stack[i]]
                    to_eval=self.unpack()
                    if to_eval:
                        oper_result=eval(str(to_eval[0])+operation+str(to_eval[1]))
                        #print(oper_result)
                        self.count_stack[-2]=[oper_result,'value']
                        self.count_stack=self.count_stack[:-1]
                    else:
                        print('ERROR: wrong types')
                        return -1
                if stack[i] in binar_operators:
                    value=self.unpack(number=1)
                    #print(self.count_stack[-2])
                    temp=[is_number(value),is_number(operands[self.count_stack[-2][0]][2])]
                    if temp[0]==temp[1]:
                        operands[self.count_stack[-2][0]][2]=value
                        self.count_stack=self.count_stack[:-2]
                if stack[i] in unar_operators:
                    value=self.unpack(number=1)
                    if stack[i]==-5 and is_number(value):
                        self.count_stack[-1]=[value*-1,'value']
                        
                    #print(oper_result)
                    
                #print('___',count_stack)
            i+=1
        return self.count_stack
    def unpack(self,number=2):
        result=[]
        if number==2:
            for i in self.count_stack[-2:]:
                if i[1]=='link':
                    result.append(operands[i[0]][2])
                    continue
                elif i[1]=='value':
                    result.append(i[0])
                    continue
                else:
                    print('Unknown type')
            print(result)
            if is_number(result[0]) and is_number(result[1]):
                return result
            else:
                return False
        if number==1:
            temp=self.count_stack[-1]
            return (operands[temp[0]][2] if self.count_stack[-1][1]=='link' else temp[0])
def is_number(x):
    return isinstance(x, (int, long, float))

In [240]:
worker=parser(stack)

In [241]:
worker.count_reversed_form(stack)

[]
[[6, 'link']]
[[6, 'link'], [2, 'link']]
[[6, 'link'], [2, 'link'], [2, 'link']]
[[6, 'link'], [2, 'link'], [2, 'link'], [6, 'link']]
[[6, 'link'], [2, 'link'], [2, 'link'], [-8, 'value']]
[2, -8]
[[6, 'link'], [2, 'link'], [-6, 'value']]
[2, -6]
[[6, 'link'], [-12, 'value']]


[]

In [235]:
operands

{1: [1, 'numb', 1],
 2: [2, 'numb', 2],
 3: [3, 'numb', 3],
 4: [4.5, 'numb', 4.5],
 5: ['some_string', 'char', 'some_string'],
 6: ['a', 'var', -12],
 7: ['b', 'var', nan]}

In [172]:
a='dsfsdf'

In [22]:
a=[[1,'a'],[2,'b'],[3,'c']]

In [24]:
for i in a[-2:]:
    print(i)

[2, 'b']
[3, 'c']


In [154]:
print(names)

{1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10.3', -1: '+', -40: 'for', -32: 'else', -31: 'if', 101: 'a', -21: 'goto', -11: 'float', -10: 'int', -8: ':', -7: '=', -6: '0', -5: nan, -4: '*', -3: '/', -2: '-'}


In [30]:
## 1 с перезаписью, заполнение при объявлении
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,101,-7,1,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5]

## 1 переменная, заполнение после объявления
#stack=[-5,-10,101,-5,101,-7,1,2,-1,4,-4,3,-2,-5]

## С goto
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,  
#        -21,30,101,-7,1,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5,30,-5,
#        101,-7,1,2,-1,6,-4,3,-2,-5]

## 2 переменные с ошибкой типов
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,-10,102,-7,101,10,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5] 

## 2 переменные заполнение при объявлении
stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,-10,102,-7,101,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5]

In [157]:
table.tail(14)

,token,value
11,-1,+
12,-2,-
13,-3,/
14,-4,*
15,-5,;
16,-6,0
17,-7,=
18,-8,:
19,-10,int
20,-11,float


In [149]:
worker.interpretate()

[{'cord': 101, 'type': 'int', 'value': 9},
 {'cord': 102, 'type': 'int', 'value': -32}]

In [42]:
a=2.5

In [43]:
b = str(type(a))

In [51]:
b.split("'")[1]

'float'

In [48]:
b.find("'")

6

In [41]:
b.__repr__

<slot wrapper '__repr__' of 'type' objects>

## (1+2)*4-3 <=> 1 2 + 4 × 3 -

In [147]:
count_reversed_form(stack,start_point=0)

([], 1)
([1], 2)
([1, 2], 3)
([1, 2, 3], -4)
([1, 6], 4)
([1, 6, 4], -4)
([1, 24], -1)
([25], 5)
([25, 5], 6)
([25, 5, 6], -2)
([25, -1], 7)
([25, -1, 7], 8)
([25, -1, 7, 8], -4)
([25, -1, 56], 9)
([25, -1, 56, 9], -1)
([25, -1, 65], -4)
([25, -65], -1)


-40

In [68]:
a=[1,2,-3,4,5]

In [83]:
exec(expression)

SyntaxError: invalid syntax (<ipython-input-83-fb2e0791accf>, line 1)

In [84]:
expression=str(names[1])+str(names[-1])+str(names[3])

In [74]:
a=exec(str('1')+str('+')+str('3'))

SyntaxError: invalid syntax (<ipython-input-74-caaaa06abadc>, line 1)

In [85]:
expression

'1+3'

In [ ]:
def interpretate(self):
        stack=self.stack
        #print(stack)
        i=0
        while i!=len(stack):
            #print(stack[i])
            if stack[i]>100:
                ## Work with variables
                #print(stack[i:])
                if stack[i-1]>-20 and stack[i-1]<=-10: ## First declaration(type before variable)
                    #print(stack[i:])
                    if self.find_value(stack[i]):  ## Check for redeclaration
                        print("ERROR: {} variable redeclaration".format(names[stack[i]]))
                        return -3
                    variable={}
                    variable['type']=names[stack[i-1]]
                    variable['cord']=stack[i]
                    if stack[i+1] not in self.end_symbols: ## some computations just after declaration
                        variable['value'],i=self.count_reversed_form(start_point=i+2)
                        if self.get_type(variable['value'])!=variable['type']:
                            print('ERROR: types mismatch:{}->{}'
                                  .format(self.get_type(variable['value']),variable['type']))
                            return -4
                        else:
                            self.user_variables.append(variable)
                        #print(self.user_variables)
                    else: ## no operations after declaration
                        variable['value']=np.NAN
                        self.user_variables.append(variable)
                else :  ## variable was already declared
                    declared_variable=self.find_value(stack[i])
                    if not declared_variable:
                        print('ERROR: variable {} was not declared'.format(names[stack[i]]))
                        return -5
                    new_value,temp=self.count_reversed_form(start_point=i+2)
                    if self.get_type(new_value)!=declared_variable['type']:
                            print('ERROR: types mismatch:{}->{}'
                                  .format(self.get_type(new_value),declared_variable['type']))
                            return -4
                    else:
                        self.change_value(stack[i],new_value)
                        i=temp
                    #print(self.user_variables)
                i+=1
                #print(stack[i])
                continue
            if stack[i]==-8:  ## goto mark
                self.check_points[stack[i+1]]=i+1
                continue
            if stack[i]==-21: ## goto
                if stack[i+1] in self.check_points.keys():
                    i=self.check_points[stack[i]]
                    continue
                else:
                    temp=stack[i+1]
                    try:
                        i=next(x for x in range(i+2,len(stack)) if stack[x]==temp)+1
                        continue
                    except:
                        print('ERROR: no mark for goto {} found'.format(names[stack[i+1]]))
#             if stack[i]==-31:
                
            i+=1
        return self.user_variables
    def find_value(self,cord):
        for i in self.user_variables:
            if i['cord']==cord:
                return i
        return False
    def change_value(self,cord,new_value):
        for i in self.user_variables:
            if i['cord']==cord:
                i['value']=new_value
                return 0
    def get_type(self,x):
        temp=str(type(x))
        return temp.split("'")[1]